In [29]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma

In [30]:
# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfgeXvuAmrfh8E-QzybfwXKx0humqS5TD54A95h-BZ7La60QwBiTX3-SYvc

Successfully saved authorization token.


In [31]:
# Define geometry for data collection

roi = ee.Geometry.Rectangle([69.98568, 22.66953, 72.76651, 24.60271]);  # Nanda Bet S1 extent
roi_small = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  # Nanda Bet small region

# boundary shapefiles
shp_file = 'data/s1_boundary.shp'
s1_boundary = geemap.shp_to_ee(shp_file)
# type(s1_boundary)

shp_file = 'data/small_region_around_nanda_bet_extent.shp'
small_boundary = geemap.shp_to_ee(shp_file)

In [37]:
# Import s1, s2, dynamic world data

d1 = '2020-01-01'
d2 = '2021-01-01'

# d1 = '2020-03-01'
# d2 = '2020-03-04'

# d1 = '2020-10-27'
# d2 = '2020-10-30'

# d1 = '2020-04-07'
# d2 = '2020-04-13'

# d1 = '2020-11-21'
# d2 = '2020-11-24'


# get Dynamic World data
dw_collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterBounds(s1_boundary)
    .filterDate(d1, d2))    
print('Number of images in dynamic world collection:', dw_collection.size().getInfo())
# print('Bands:', dw_collection.first().bandNames().getInfo())


# get Sentinel-2 data
s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(s1_boundary).filterDate(d1, d2)  
print('Number of images in s2 collection:', s2_collection.size().getInfo())
# print('Bands:', s2_collection.first().bandNames().getInfo())


# get Sentinel-1 data
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                 .filter(ee.Filter.date(d1,d2))
                 .filterBounds(roi_small))
print('Number of images in s1 collection:', s1_collection.size().getInfo())

Number of images in s1 collection: 31


In [38]:
# get dates of images

acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

['01-03-2020', '01-15-2020', '01-27-2020', '02-08-2020', '02-20-2020', '03-03-2020', '03-15-2020', '03-27-2020', '04-08-2020', '04-20-2020', '05-02-2020', '05-14-2020', '05-26-2020', '06-07-2020', '06-19-2020', '07-01-2020', '07-13-2020', '07-25-2020', '08-06-2020', '08-18-2020', '08-30-2020', '09-11-2020', '09-23-2020', '10-05-2020', '10-17-2020', '10-29-2020', '11-10-2020', '11-22-2020', '12-04-2020', '12-16-2020', '12-28-2020']


In [39]:
# define function to implement speckle filter and get vv/vh ratio
def speckle(img):
    from speckle_filter import leesigma
    k = 5
    vv = img.select('VV')
    vh = img.select('VH')
    vv_vh = vv.divide(vh).rename('VV_VH')
    vv_filtered = leesigma(vv,k).rename('VV_filtered')
    vh_filtered = leesigma(vh,k).rename('VH_filtered')
    vv_vh_filtered = vv_filtered.divide(vh_filtered).rename('VV_VH_filtered')
    img = img.addBands(ee.Image([vv_vh, vv_filtered, vh_filtered, vv_vh_filtered]))
    return img

s1_collection = s1_collection.map(speckle)
# print(s1_collection.first().bandNames().getInfo())

In [172]:
# get bands of interest

# dynamic world
water = dw_collection.select('water').mean().clip(s1_boundary).rename('water');
label = dw_collection.select('label').mode().clip(s1_boundary).rename('label');
flooded_vegetation = dw_collection.select('flooded_vegetation').mean().clip(s1_boundary).rename('flooded_vegetation');
bare = dw_collection.select('bare').mean().clip(s1_boundary).rename('bare');
trees = dw_collection.select('water').mean().clip(s1_boundary).rename('trees');
grass = dw_collection.select('water').mean().clip(s1_boundary).rename('grass');
crops = dw_collection.select('water').mean().clip(s1_boundary).rename('crops');
shrub_and_scrub = dw_collection.select('water').mean().clip(s1_boundary).rename('shrub_and_scrub');
built = dw_collection.select('water').mean().clip(s1_boundary).rename('built');
snow_and_ice = dw_collection.select('water').mean().clip(s1_boundary).rename('snow_and_ice');
dw_img = water.addBands(ee.Image([bare,flooded_vegetation,trees,grass,crops,shrub_and_scrub,built,snow_and_ice]))


# s2
B2 = s2_collection.select('B2').mean().clip(s1_boundary).rename('B2').divide(10000);
B3 = s2_collection.select('B3').mean().clip(s1_boundary).rename('B3').divide(10000);
B4 = s2_collection.select('B4').mean().clip(s1_boundary).rename('B4').divide(10000);
B8 = s2_collection.select('B8').mean().clip(s1_boundary).rename('B8').divide(10000);
s2_img = B2.addBands(ee.Image([B3,B4,B8]))


# s1
s1_img = s1_collection.select(['VV_filtered','VH_filtered','VV_VH_filtered'])




# define visibility parameters
CLASS_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'];

VIS_PALETTE = [
    '419BDF', '397D49', '88B053', '7A87C6',
    'E49635', 'DFC35A', 'C4281B', 'A59B8F',
    'B39FE1'];
    
label_vis = {
  'min': 0,
  'max': 8,
  'palette': VIS_PALETTE,
  'opacity': 1
}

water_vis = {
  'min': 0,
  'max': 0.8,
  'palette': ['ffffff', '0000ff'],
  'opacity': 1
}

flooded_veg_vis = {
  'min': 0,
  'max': 0.5,
  'palette': ['ffffff', 'ff0000'],
  'opacity': 1
}

bare_vis = {
  'min': 0,
  'max': 0.8,
  'palette': ['ffffff', '990022'],
  'opacity': 0.5
}

s2_vis = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
};

vv_vis = {
    'min': 0,
    'max': 0.2,
    'palette': ['#000000','#ffffff']
}

vh_vis = {
    'min': 0,
    'max': 0.03,
    'palette': ['#000000','#ffffff']
}

vv_vh_vis = {
    'min': 3,
    'max': 30,
    'palette': ['#000000','#ffffff']
}



In [174]:
print(dw_img.bandNames().getInfo())

['water', 'bare', 'flooded_vegetation', 'trees', 'grass', 'crops', 'shrub_and_scrub', 'built', 'snow_and_ice']


In [140]:
M = geemap.Map()
M.centerObject(roi, 8)
M.add_basemap('HYBRID')

M.addLayer(dw_img.select('water'), water_vis, 'water');
M.addLayer(dw_img.select('flooded_vegetation'), flooded_veg_vis, 'flooded vegetation');
M.addLayer(dw_img.select('bare'), bare_vis, 'bare ground', 1);
M.addLayer(s2_img, s2_vis, 's2')
M.addLayer(s1_img.select('VV_filtered'), vv_vis, 's1')

M.addLayer(s1_boundary,{},'boundary')
M  # show map

Map(center=[23.639935260097975, 71.37609499999995], controls=(WidgetControl(options=['position', 'transparent_…

In [141]:
txt = d1 + '_' + d2
txt2 = 's2_' + txt
print(txt2)

s2_2020-03-01_2020-03-04


In [143]:
task_id = []
txt = d1 + '_' + d2


# export dynamic world data
txt2 = 'dynamic_world_bands_' + txt
task = ee.batch.Export.image.toDrive(image=dw_img,
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)


txt2 = 'dynamic_world_label_' + txt
task = ee.batch.Export.image.toDrive(image=label,
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)


# export s2 data
txt2 = 's2_' + txt
task = ee.batch.Export.image.toDrive(image=s2_img,
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)


# export s1 data
txt2 = 's1_' + txt
task = ee.batch.Export.image.toDrive(image=s1_img.first(),
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)

In [171]:
task.list()

[<Task K3GFIJG44GOVNCYMI55M6L3E EXPORT_IMAGE: s2_b12_2020-11-21_2020-11-24 (READY)>,
 <Task GAGFL5QC6IIAWNKC6FPVMHQF EXPORT_IMAGE: s2_b12_2020-04-07_2020-04-13 (READY)>,
 <Task IRQBXCDR5QQZDYJJBCE5Q5ZE EXPORT_IMAGE: s2_b12_2020-10-27_2020-10-30 (RUNNING)>,
 <Task MWVUTGBLMVEHNGJEFP6E5URF EXPORT_IMAGE: s2_b12_2020-03-01_2020-03-04 (RUNNING)>,
 <Task USXKAPETAJOD7EHW2LB2KF57 EXPORT_IMAGE: s2_b11_2020-11-21_2020-11-24 (COMPLETED)>,
 <Task NRRBY4GAD4YZDX2W7BDTD32B EXPORT_IMAGE: s2_b11_2020-04-07_2020-04-13 (COMPLETED)>,
 <Task 2ZSRC5GX2SU2LHA3UZXQS26R EXPORT_IMAGE: s2_b11_2020-10-27_2020-10-30 (COMPLETED)>,
 <Task LOOM5TSJ2VSONVD226D6KZ2C EXPORT_IMAGE: s2_b11_2020-03-01_2020-03-04 (COMPLETED)>,
 <Task LIYJNXGTZDYULYSJGTJEH32H EXPORT_IMAGE: s1_2020-11-21_2020-11-24 (COMPLETED)>,
 <Task 6KMMYK2E5JD4JVFVDH3XPPWN EXPORT_IMAGE: s2_2020-11-21_2020-11-24 (COMPLETED)>,
 <Task SY4B7ANJRA4362R75I2FRVYU EXPORT_IMAGE: dynamic_world_label_2020-11-21_2020-11-24 (COMPLETED)>,
 <Task D25GWYS76SJ4ITD5N4YIG

In [47]:
#task.status()
t_id = "BVVGMAO2RA54YQE4BZ3P5PE4"
print(ee.data.getTaskStatus(t_id))

[{'state': 'RUNNING', 'description': 'dynamic_world_bands_2020_0301-0304', 'creation_timestamp_ms': 1666307095868, 'update_timestamp_ms': 1666307597883, 'start_timestamp_ms': 1666307116300, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'BVVGMAO2RA54YQE4BZ3P5PE4', 'name': 'projects/earthengine-legacy/operations/BVVGMAO2RA54YQE4BZ3P5PE4'}]


In [150]:
# Import s1, s2, dynamic world data
# d1 = '2020-03-01'
# d2 = '2020-03-04'

# d1 = '2020-10-27'
# d2 = '2020-10-30'

# d1 = '2020-04-07'
# d2 = '2020-04-13'

# d1 = '2020-11-21'
# d2 = '2020-11-24'

d1 = '2020-01-15'
d2 = '2020-01-17'


# get Dynamic World data
dw_collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
    .filterBounds(s1_boundary)
    .filterDate(d1, d2))    
#print('Number of images in dynamic world collection:', dw_collection.size().getInfo())
# print('Bands:', dw_collection.first().bandNames().getInfo())


# get Sentinel-2 data
s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(s1_boundary).filterDate(d1, d2)  
#print('Number of images in s2 collection:', s2_collection.size().getInfo())
# print('Bands:', s2_collection.first().bandNames().getInfo())


# get Sentinel-1 data
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                 .filter(ee.Filter.date(d1,d2))
                 .filterBounds(roi_small))
#print('Number of images in s1 collection:', s1_collection.size().getInfo())
s1_collection = s1_collection.map(speckle)


water = dw_collection.select('water').mean().clip(s1_boundary).rename('water');
label = dw_collection.select('label').mode().clip(s1_boundary).rename('label');
flooded_vegetation = dw_collection.select('flooded_vegetation').mean().clip(s1_boundary).rename('flooded_vegetation');
bare = dw_collection.select('bare').mean().clip(s1_boundary).rename('bare');
trees = dw_collection.select('water').mean().clip(s1_boundary).rename('trees');
grass = dw_collection.select('water').mean().clip(s1_boundary).rename('grass');
crops = dw_collection.select('water').mean().clip(s1_boundary).rename('crops');
shrub_and_scrub = dw_collection.select('water').mean().clip(s1_boundary).rename('shrub_and_scrub');
built = dw_collection.select('water').mean().clip(s1_boundary).rename('built');
snow_and_ice = dw_collection.select('water').mean().clip(s1_boundary).rename('snow_and_ice');
dw_img = water.addBands(ee.Image([bare,flooded_vegetation,trees,grass,crops,shrub_and_scrub,built,snow_and_ice]))


# s2
B2 = s2_collection.select('B2').mean().clip(s1_boundary).rename('B2').divide(10000);
B3 = s2_collection.select('B3').mean().clip(s1_boundary).rename('B3').divide(10000);
B4 = s2_collection.select('B4').mean().clip(s1_boundary).rename('B4').divide(10000);
B8 = s2_collection.select('B8').mean().clip(s1_boundary).rename('B8').divide(10000);
s2_img = B2.addBands(ee.Image([B3,B4,B8]))


# s1
s1_img = s1_collection.select(['VV_filtered','VH_filtered','VV_VH_filtered'])



task_id = []
txt = d1 + '_' + d2


# export dynamic world data
txt2 = 'dynamic_world_bands_' + txt
task = ee.batch.Export.image.toDrive(image=dw_img,
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)


txt2 = 'dynamic_world_label_' + txt
task = ee.batch.Export.image.toDrive(image=label,
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)


# export s2 data
txt2 = 's2_' + txt
task = ee.batch.Export.image.toDrive(image=s2_img,
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)


# export s1 data
txt2 = 's1_' + txt
task = ee.batch.Export.image.toDrive(image=s1_img.first(),
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)

In [170]:
# Import individual s2 bands

# d1 = '2020-03-01'
# d2 = '2020-03-04'

# d1 = '2020-10-27'
# d2 = '2020-10-30'

# d1 = '2020-04-07'
# d2 = '2020-04-13'

# d1 = '2020-11-21'
# d2 = '2020-11-24'


# get Sentinel-2 data
s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(s1_boundary).filterDate(d1, d2)  
#print('Number of images in s2 collection:', s2_collection.size().getInfo())
# print('Bands:', s2_collection.first().bandNames().getInfo())


# s2 bands
# B11 = s2_collection.select('B11').mean().clip(s1_boundary).rename('B11').divide(10000); # SWIR1
B12 = s2_collection.select('B12').mean().clip(s1_boundary).rename('B12').divide(10000); # SWIR2


# export s2 data
task_id = []
txt = d1 + '_' + d2
txt2 = 's2_b12_' + txt
task = ee.batch.Export.image.toDrive(image=B12,
                                     description=txt2,
                                     scale=100,
                                     fileNamePrefix=txt2,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF',
                                     maxPixels=2e10,
                                     region=s1_boundary.geometry())
task.start()
task_id.append(task.id)



In [166]:
print(s2_collection.first().bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


In [27]:
# Import s1, s2, dynamic world data in a loop

dates_beg = ['2020-01-15','2020-01-26','2020-02-07','2020-02-20','2020-03-01','2020-04-07','2020-04-30','2020-05-12',
             '2020-05-25','2020-09-22','2020-10-27','2020-11-08','2020-11-21','2020-12-03']
             
dates_end = ['2020-01-17','2020-01-29','2020-02-11','2020-02-23','2020-03-05','2020-04-14','2020-05-03','2020-05-16',
             '2020-05-28','2020-09-25','2020-10-31','2020-11-12','2020-11-25','2020-12-07']

n = len(dates_beg)

for idx in range(n):
    d1 = dates_beg[idx]
    d2 = dates_end[idx]
    
#     # get Dynamic World data
#     dw_collection = (ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1")
#         .filterBounds(s1_boundary)
#         .filterDate(d1, d2))    

    # get Sentinel-2 data
    s2_collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(s1_boundary).filterDate(d1, d2)  

#     # get Sentinel-1 data
#     s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
#                      .filter(ee.Filter.eq('instrumentMode', 'IW'))
#                      .filterMetadata('resolution_meters', 'equals', 10)
#                      .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
#                      .filter(ee.Filter.date(d1,d2))
#                      .filterBounds(roi_small))
#     s1_collection = s1_collection.map(speckle)


#     water = dw_collection.select('water').mean().clip(s1_boundary).rename('water');
#     label = dw_collection.select('label').mode().clip(s1_boundary).rename('label');
#     flooded_vegetation = dw_collection.select('flooded_vegetation').mean().clip(s1_boundary).rename('flooded_vegetation');
#     bare = dw_collection.select('bare').mean().clip(s1_boundary).rename('bare');
#     trees = dw_collection.select('water').mean().clip(s1_boundary).rename('trees');
#     grass = dw_collection.select('water').mean().clip(s1_boundary).rename('grass');
#     crops = dw_collection.select('water').mean().clip(s1_boundary).rename('crops');
#     shrub_and_scrub = dw_collection.select('water').mean().clip(s1_boundary).rename('shrub_and_scrub');
#     built = dw_collection.select('water').mean().clip(s1_boundary).rename('built');
#     snow_and_ice = dw_collection.select('water').mean().clip(s1_boundary).rename('snow_and_ice');
#     dw_img = water.addBands(ee.Image([bare,flooded_vegetation,trees,grass,crops,shrub_and_scrub,built,snow_and_ice]))


    # s2
    B2 = s2_collection.select('B2').mean().clip(s1_boundary).rename('B2').divide(10000);
    B3 = s2_collection.select('B3').mean().clip(s1_boundary).rename('B3').divide(10000);
    B4 = s2_collection.select('B4').mean().clip(s1_boundary).rename('B4').divide(10000);
    B8 = s2_collection.select('B8').mean().clip(s1_boundary).rename('B8').divide(10000);
    B11 = s2_collection.select('B11').mean().clip(s1_boundary).rename('B11').divide(10000); # SWIR1
    B12 = s2_collection.select('B12').mean().clip(s1_boundary).rename('B12').divide(10000); # SWIR2
    s2_img = B2.addBands(ee.Image([B3,B4,B8,B11,B12]))


#     # s1
#     s1_img = s1_collection.select(['VV_filtered','VH_filtered','VV_VH_filtered'])



    task_id = []
    txt = d1 + '_' + d2


#     # export dynamic world data
#     txt2 = 'dynamic_world_bands_' + txt
#     task = ee.batch.Export.image.toDrive(image=dw_img,
#                                          description=txt2,
#                                          scale=100,
#                                          fileNamePrefix=txt2,
#                                          crs='EPSG:4326',
#                                          fileFormat='GeoTIFF',
#                                          maxPixels=2e10,
#                                          region=s1_boundary.geometry())
#     task.start()
#     task_id.append(task.id)


#     txt2 = 'dynamic_world_label_' + txt
#     task = ee.batch.Export.image.toDrive(image=label,
#                                          description=txt2,
#                                          scale=100,
#                                          fileNamePrefix=txt2,
#                                          crs='EPSG:4326',
#                                          fileFormat='GeoTIFF',
#                                          maxPixels=2e10,
#                                          region=s1_boundary.geometry())
#     task.start()
#     task_id.append(task.id)


    # export s2 data
    txt2 = 's2_' + txt
    task = ee.batch.Export.image.toDrive(image=s2_img,
                                         description=txt2,
                                         scale=100,
                                         fileNamePrefix=txt2,
                                         crs='EPSG:4326',
                                         fileFormat='GeoTIFF',
                                         maxPixels=2e10,
                                         region=s1_boundary.geometry())
    task.start()
    task_id.append(task.id)


#     # export s1 data
#     txt2 = 's1_' + txt
#     task = ee.batch.Export.image.toDrive(image=s1_img.first(),
#                                          description=txt2,
#                                          scale=100,
#                                          fileNamePrefix=txt2,
#                                          crs='EPSG:4326',
#                                          fileFormat='GeoTIFF',
#                                          maxPixels=2e10,
#                                          region=s1_boundary.geometry())
#     task.start()
#     task_id.append(task.id)
    
    print(d1, ' ', d2)    

2020-03-01   2020-03-05
2020-04-07   2020-04-14
2020-10-27   2020-10-31
2020-11-21   2020-11-25


In [49]:
task.list()

[<Task NTEVFW7QUBFZ73RX5LU3L34H EXPORT_IMAGE: s1_nanda_bet_small_12-28-2020 (READY)>,
 <Task M3Y36RGQ7KVLAFVN3BZEP632 EXPORT_IMAGE: s1_nanda_bet_small_12-16-2020 (READY)>,
 <Task KMHEVAEACLM4U2TQNCKQTJCU EXPORT_IMAGE: s1_nanda_bet_small_12-04-2020 (READY)>,
 <Task BVKQ4IFLIVVMS4K5573Q73JH EXPORT_IMAGE: s1_nanda_bet_small_11-22-2020 (READY)>,
 <Task LMSJQQ6NG6OXORLFJFUUN633 EXPORT_IMAGE: s1_nanda_bet_small_11-10-2020 (READY)>,
 <Task QQVUTS5GADO4FAFAHYCZ5KXR EXPORT_IMAGE: s1_nanda_bet_small_10-29-2020 (READY)>,
 <Task UEAPLYX7WME2JOJKDAFJGBLC EXPORT_IMAGE: s1_nanda_bet_small_10-17-2020 (READY)>,
 <Task YFO766SHE2PL2J3KH42WAQID EXPORT_IMAGE: s1_nanda_bet_small_10-05-2020 (READY)>,
 <Task 5TLZ3PUTOBBUDYEQ7XN3KDZE EXPORT_IMAGE: s1_nanda_bet_small_09-23-2020 (READY)>,
 <Task O5IO3KB3C4QHXSZZLDYA55DO EXPORT_IMAGE: s1_nanda_bet_small_09-11-2020 (READY)>,
 <Task IGGKRKCFWMT6UOMLU66Y5Y6B EXPORT_IMAGE: s1_nanda_bet_small_08-30-2020 (READY)>,
 <Task ZDTMUETJMAWPMDOFRQFZKEV2 EXPORT_IMAGE: s1_nanda

In [45]:
n = s1_collection.size().getInfo()
s1_collection_list = s1_collection.toList(n)
print(n)

31


In [46]:

acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

['01-03-2020', '01-15-2020', '01-27-2020', '02-08-2020', '02-20-2020', '03-03-2020', '03-15-2020', '03-27-2020', '04-08-2020', '04-20-2020', '05-02-2020', '05-14-2020', '05-26-2020', '06-07-2020', '06-19-2020', '07-01-2020', '07-13-2020', '07-25-2020', '08-06-2020', '08-18-2020', '08-30-2020', '09-11-2020', '09-23-2020', '10-05-2020', '10-17-2020', '10-29-2020', '11-10-2020', '11-22-2020', '12-04-2020', '12-16-2020', '12-28-2020']


In [47]:
ns = ee.Image(s1_collection_list.get(1)).select('VV_filtered').projection().nominalScale().getInfo()
print(ns)

10


In [48]:
# export s1 data in small region around nanda bet
task_id = []

for i in range(0,n):
    img = ee.Image(s1_collection_list.get(i)).select(['VV_filtered','VH_filtered','VV_VH_filtered'])
    txt = 's1_nanda_bet_small_' + date_list[i]
    task = ee.batch.Export.image.toDrive(image=img,
                                        description=txt,
                                        scale=ns,
                                        region=small_boundary.geometry(),
                                        fileNamePrefix=txt,
                                        crs='EPSG:4326',
                                        fileFormat='GeoTIFF',
                                        maxPixels=1e9)
    task.start()
    task_id.append(task.id)
    print(date_list[i])
    


01-03-2020
01-15-2020
01-27-2020
02-08-2020
02-20-2020
03-03-2020
03-15-2020
03-27-2020
04-08-2020
04-20-2020
05-02-2020
05-14-2020
05-26-2020
06-07-2020
06-19-2020
07-01-2020
07-13-2020
07-25-2020
08-06-2020
08-18-2020
08-30-2020
09-11-2020
09-23-2020
10-05-2020
10-17-2020
10-29-2020
11-10-2020
11-22-2020
12-04-2020
12-16-2020
12-28-2020


In [54]:
task.list()

[<Task NTEVFW7QUBFZ73RX5LU3L34H EXPORT_IMAGE: s1_nanda_bet_small_12-28-2020 (COMPLETED)>,
 <Task M3Y36RGQ7KVLAFVN3BZEP632 EXPORT_IMAGE: s1_nanda_bet_small_12-16-2020 (COMPLETED)>,
 <Task KMHEVAEACLM4U2TQNCKQTJCU EXPORT_IMAGE: s1_nanda_bet_small_12-04-2020 (COMPLETED)>,
 <Task BVKQ4IFLIVVMS4K5573Q73JH EXPORT_IMAGE: s1_nanda_bet_small_11-22-2020 (COMPLETED)>,
 <Task LMSJQQ6NG6OXORLFJFUUN633 EXPORT_IMAGE: s1_nanda_bet_small_11-10-2020 (COMPLETED)>,
 <Task QQVUTS5GADO4FAFAHYCZ5KXR EXPORT_IMAGE: s1_nanda_bet_small_10-29-2020 (COMPLETED)>,
 <Task UEAPLYX7WME2JOJKDAFJGBLC EXPORT_IMAGE: s1_nanda_bet_small_10-17-2020 (COMPLETED)>,
 <Task YFO766SHE2PL2J3KH42WAQID EXPORT_IMAGE: s1_nanda_bet_small_10-05-2020 (COMPLETED)>,
 <Task 5TLZ3PUTOBBUDYEQ7XN3KDZE EXPORT_IMAGE: s1_nanda_bet_small_09-23-2020 (COMPLETED)>,
 <Task O5IO3KB3C4QHXSZZLDYA55DO EXPORT_IMAGE: s1_nanda_bet_small_09-11-2020 (COMPLETED)>,
 <Task IGGKRKCFWMT6UOMLU66Y5Y6B EXPORT_IMAGE: s1_nanda_bet_small_08-30-2020 (COMPLETED)>,
 <Task ZDT